In [1]:
!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

Looking in links: http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html
  The repository located at h2o-release.s3.amazonaws.com is not a trusted or secure host and is being ignored. If this repository is available via HTTPS we recommend you use HTTPS instead, otherwise you may silence this warning and allow it anyway with '--trusted-host h2o-release.s3.amazonaws.com'.
    100% |████████████████████████████████| 120.9MB 199kB/s 
  The repository located at h2o-release.s3.amazonaws.com is not a trusted or secure host and is being ignored. If this repository is available via HTTPS we recommend you use HTTPS instead, otherwise you may silence this warning and allow it anyway with '--trusted-host h2o-release.s3.amazonaws.com'.
  Stored in directory: /root/.cache/pip/wheels/f4/21/59/e953dbf8206d5061ce1346e119dad9ffafd7f88011430682ba
Successfully built h2o


In [34]:
import h2o
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from h2o.automl import H2OAutoML

h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,19 mins 30 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.1.6
H2O cluster version age:,10 days
H2O cluster name:,H2O_from_python_unknownUser_bzwy0k
H2O cluster total nodes:,1
H2O cluster free memory:,2.988 Gb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"locked, healthy"


In [0]:
from sklearn.metrics import accuracy_score

In [0]:
scores= pd.read_csv('combinedall.csv')

In [37]:
scores.head(5)

,Unnamed: 0,AWSSentiment,AWSMixed,AWSNegative,AWSNeutral,AWSPositive,GCPmagnitude,GCPscore,IBMlabel,IBMscore,id,Azurescore,realSentiment,text
0,1,POSITIVE,0.003024,0.003904,0.201138,0.791934,0.0,0.0,positive,0.987951,1,0.887542,positive,"Thank you, Ellen. We have a strong 2018, with ..."
1,2,NEUTRAL,0.002820,0.006084,0.974791,0.016305,0.7,0.1,positive,0.655981,2,0.832603,neutral,Let me now turn to and talk about Other Bets. ...
2,3,NEUTRAL,0.001650,0.008785,0.940597,0.048969,0.4,0.1,neutral,0.000000,3,0.500000,neutral,"Secondly, could you talk about advertising rev..."
3,4,NEUTRAL,0.003136,0.004265,0.859948,0.132651,0.3,0.1,positive,0.580306,4,0.792194,positive,Our Investment Banking franchise overall remai...
4,5,NEUTRAL,0.006692,0.294444,0.688630,0.010235,0.3,0.0,positive,0.683194,5,0.749581,neutral,Great. Thank you. I just wanted to follow up o...


In [0]:
coloumns=['AWSMixed','AWSNegative','AWSNeutral','AWSPositive','GCPscore','IBMscore','Azurescore']

In [0]:
X=np.asarray(scores[coloumns])
y=np.asarray(scores['realSentiment'])

In [0]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)

In [41]:
x_train.shape

(800, 7)

In [0]:
aml = H2OAutoML(max_models=20, seed=1)

In [0]:
df = pd.DataFrame(x_train)
df['realSentiment'] = y_train

In [44]:
h2o_data = h2o.import_file('combinedall.csv')

predictors = ['AWSMixed','AWSNegative','AWSNeutral','AWSPositive','GCPscore','IBMscore','Azurescore']
response = "realSentiment"

train, valid = h2o_data.split_frame(ratios = [.8], seed = 11)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [45]:
aml.train(x=predictors, y=response, training_frame=h2o_data)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [47]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,mean_per_class_error,logloss,rmse,mse
XGBoost_3_AutoML_20190324_041503,0.6481,0.961656,0.609975,0.37207
DeepLearning_grid_1_AutoML_20190324_041503_model_1,0.654946,1.11764,0.589651,0.347688
DeepLearning_1_AutoML_20190324_041503,0.656802,1.13193,0.609633,0.371653
XGBoost_2_AutoML_20190324_041503,0.658342,0.973777,0.613946,0.37693
DeepLearning_grid_1_AutoML_20190324_041503_model_2,0.658444,0.978294,0.583066,0.339966
XGBoost_grid_1_AutoML_20190324_041503_model_3,0.658798,0.949927,0.601113,0.361337
GBM_3_AutoML_20190324_041503,0.663433,0.990539,0.601853,0.362227
GLM_grid_1_AutoML_20190324_041503_model_1,0.666667,0.924414,0.582357,0.33914
XGBoost_1_AutoML_20190324_041503,0.666743,0.955444,0.601382,0.36166
XRT_1_AutoML_20190324_041503,0.667474,3.50797,0.624007,0.389384


In [49]:
predictions = aml.leader.predict(valid)

xgboost prediction progress: |████████████████████████████████████████████| 100%


In [51]:
predictions

predict,negative,neutral,positive
positive,0.157612,0.359932,0.482456
positive,0.140292,0.411337,0.448371
positive,0.190692,0.399847,0.409461
positive,0.239194,0.295844,0.464962
positive,0.177064,0.384198,0.438738
positive,0.152599,0.355678,0.491723
positive,0.178001,0.341682,0.480317
neutral,0.1961,0.405548,0.398353
positive,0.172433,0.387065,0.440502
positive,0.191229,0.343165,0.465606


In [52]:
metrics = aml.leader.model_performance(valid)
metrics


ModelMetricsMultinomial: xgboost
** Reported on test data. **

MSE: 0.3301899617415586
RMSE: 0.5746215813398925
LogLoss: 0.8678709433116387
Mean Per-Class Error: 0.5454908539020689
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



negative,neutral,positive,Error,Rate
0.0,6.0,17.0,1.0,23 / 23
0.0,32.0,46.0,0.5897436,46 / 78
0.0,5.0,102.0,0.0467290,5 / 107
0.0,43.0,165.0,0.3557692,74 / 208


Top-3 Hit Ratios: 


k,hit_ratio
1,0.6442308
2,0.8942308
3,1.0


In [53]:
metrics.confusion_matrix()

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



negative,neutral,positive,Error,Rate
0.0,6.0,17.0,1.0,23 / 23
0.0,32.0,46.0,0.5897436,46 / 78
0.0,5.0,102.0,0.0467290,5 / 107
0.0,43.0,165.0,0.3557692,74 / 208
